In [2]:
import os, glob, nibabel as nib, subprocess, numpy as np, matplotlib.pyplot as plt, pandas as pd, matplotlib as mpl, nilearn
import matplotlib.cm as cm, seaborn as sns, matplotlib
from nilearn import plotting
import subprocess, numpy as np, nibabel as nib
from sklearn.metrics import pairwise_distances
import sys
sys.path.append("./mapalign./mapalign")
import mapalign
from mapalign import embed
from PIL import Image
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn import metrics
import numpy as np

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
emb = np.load('embedding_dense_DNtoWholeBrain_embRandL.npy')
emb.shape

(339, 15)

In [6]:
res = np.load('embedding_dense_DNtoWholeBrain_resRandL.npy')
res.shape

()

In [7]:
# emb shows 9 gradients. The first gradient is emb[:,0]
emb[:,0]

array([-2.13401557e-01, -1.79650569e-02, -1.85051904e-01, -2.59019818e-02,
        2.46016765e-01, -9.02283005e-02,  1.62496067e-01,  3.13953386e-01,
       -3.74847972e-02,  1.79097304e-01,  2.80331046e-01,  1.79244291e-01,
        2.50206059e-01,  2.50272500e-01,  1.58651470e-01, -2.93158009e-02,
        3.58703887e-03,  1.05955023e-01,  7.56396937e-02, -4.70052599e-02,
        2.87875460e-02,  7.24464084e-02, -1.25047532e-01, -1.13303886e-01,
        3.37818115e-02, -5.57704398e-02, -1.22737467e-01, -9.25457568e-02,
       -1.97561182e-01, -3.22733887e-01, -2.25138125e-01, -3.11740208e-01,
       -1.39199368e-01, -2.08205421e-01, -1.17342076e-01, -1.66715346e-01,
       -1.66715346e-01, -1.66715346e-01, -1.66715346e-01, -1.66715346e-01,
       -1.66715346e-01, -1.66715346e-01, -1.66715346e-01, -1.66715346e-01,
       -1.66715346e-01, -1.66715346e-01, -1.66715346e-01,  1.58784729e-01,
        1.97263143e-01,  2.31969191e-01,  8.73910424e-02, -1.31089138e-02,
       -6.16587315e-03,  

In [16]:
# build pandas dataframe for gradient 1 and 2 values for each seed
df = pd.DataFrame(index=np.arange(1, 340)) # list of seeds, from 1 to 339 (this does not correspond to the number that sheeba sent)
df['gradient1'] = emb[:,0]
df['gradient2'] = emb[:,1]
df['gradient3'] = emb[:,2]
df['gradient4'] = emb[:,3]
df['gradient5'] = emb[:,4]
df['gradient6'] = emb[:,5]
# load coordinates that sheeba sent
# THIS INCLUDES LEFT AND THEN RIGHT:
coordianteslist = [[-10,-50,-34], [-10,-50,-36], [-10,-62,-32], [-10,-62,-34], [-10,-62,-36], [-10,-64,-32], [-10,-64,-34], [-10,-64,-36], [-10,-66,-32], [-10,-66,-34], [-10,-66,-36], [-10,-68,-32], [-10,-68,-34], [-10,-68,-36], [-12,-48,-38], [-12,-50,-34], [-12,-50,-36], [-12,-50,-38], [-12,-50,-40], [-12,-52,-34], [-12,-52,-36], [-12,-52,-38], [-12,-54,-32], [-12,-54,-34], [-12,-54,-36], [-12,-56,-32], [-12,-56,-34], [-12,-56,-36], [-12,-58,-32], [-12,-58,-34], [-12,-60,-32], [-12,-60,-34], [-12,-60,-36], [-12,-62,-32], [-12,-62,-34], [-12,-62,-36], [-12,-62,-38], [-12,-62,-40], [-12,-64,-32], [-12,-64,-34], [-12,-64,-36], [-12,-64,-38], [-12,-64,-40], [-12,-66,-32], [-12,-66,-34], [-12,-66,-36], [-12,-66,-38], [-12,-68,-34], [-12,-68,-36], [-12,-68,-38], [-14,-50,-34], [-14,-50,-36], [-14,-50,-38], [-14,-50,-40], [-14,-52,-34], [-14,-52,-36], [-14,-52,-38], [-14,-52,-40], [-14,-54,-32], [-14,-54,-34], [-14,-54,-36], [-14,-54,-38], [-14,-54,-40], [-14,-56,-32], [-14,-56,-34], [-14,-56,-36], [-14,-56,-38], [-14,-56,-40], [-14,-58,-32], [-14,-58,-34], [-14,-58,-36], [-14,-58,-38], [-14,-58,-40], [-14,-60,-32], [-14,-60,-34], [-14,-60,-36], [-14,-60,-38], [-14,-60,-40], [-14,-62,-32], [-14,-62,-34], [-14,-62,-36], [-14,-62,-38], [-14,-62,-40], [-14,-64,-32], [-14,-64,-34], [-14,-64,-36], [-14,-64,-38], [-14,-64,-40], [-14,-66,-32], [-14,-66,-34], [-14,-66,-36], [-14,-66,-38], [-14,-68,-34], [-14,-68,-36], [-14,-68,-38], [-16,-50,-38], [-16,-52,-34], [-16,-52,-36], [-16,-52,-38], [-16,-52,-40], [-16,-54,-32], [-16,-54,-34], [-16,-54,-36], [-16,-54,-38], [-16,-54,-40], [-16,-56,-32], [-16,-56,-34], [-16,-56,-36], [-16,-56,-38], [-16,-56,-40], [-16,-58,-32], [-16,-58,-34], [-16,-58,-36], [-16,-58,-38], [-16,-58,-40], [-16,-60,-32], [-16,-60,-34], [-16,-60,-36], [-16,-60,-38], [-16,-60,-40], [-16,-62,-32], [-16,-62,-34], [-16,-62,-36], [-16,-62,-38], [-16,-62,-40], [-16,-64,-32], [-16,-64,-34], [-16,-64,-36], [-16,-64,-38], [-16,-64,-40], [-16,-66,-34], [-16,-66,-36], [-16,-66,-38], [-18,-52,-36], [-18,-52,-38], [-18,-52,-40], [-18,-54,-34], [-18,-54,-36], [-18,-54,-38], [-18,-54,-40], [-18,-56,-32], [-18,-56,-34], [-18,-56,-36], [-18,-56,-38], [-18,-56,-40], [-18,-58,-32], [-18,-58,-34], [-18,-58,-36], [-18,-58,-38], [-18,-58,-40], [-18,-60,-32], [-18,-60,-34], [-18,-60,-36], [-18,-60,-38], [-18,-60,-40], [-18,-62,-32], [-18,-62,-34], [-18,-62,-36], [-18,-62,-38], [-18,-62,-40], [-18,-64,-34], [-18,-64,-36], [-18,-64,-38], [-18,-64,-40], [-18,-66,-36], [-18,-66,-38], [10,-62,-30],[10,-62,-32],[10,-62,-34],[10,-64,-30],[10,-64,-32],[10,-64,-34],[10,-64,-36],[10,-66,-30],[10,-66,-32],[10,-66,-34],[12,-50,-34],[12,-50,-36],[12,-50,-38],[12,-52,-32],[12,-52,-34],[12,-62,-30],[12,-62,-32],[12,-62,-34],[12,-62,-36],[12,-64,-30],[12,-64,-32],[12,-64,-34],[12,-64,-36],[12,-64,-38],[12,-66,-30],[12,-66,-32],[12,-66,-34],[12,-66,-36],[12,-66,-38],[12,-68,-32],[12,-68,-34],[12,-68,-36],[12,-68,-38],[14,-48,-38],[14,-50,-34],[14,-50,-36],[14,-50,-38],[14,-50,-40],[14,-52,-32],[14,-52,-34],[14,-52,-36],[14,-52,-38],[14,-52,-40],[14,-54,-30],[14,-54,-32],[14,-54,-34],[14,-54,-36],[14,-54,-38],[14,-56,-30],[14,-56,-32],[14,-56,-34],[14,-56,-36],[14,-58,-30],[14,-58,-32],[14,-58,-34],[14,-60,-30],[14,-60,-32],[14,-60,-34],[14,-60,-36],[14,-62,-30],[14,-62,-32],[14,-62,-34],[14,-62,-36],[14,-62,-38],[14,-62,-40],[14,-64,-30],[14,-64,-32],[14,-64,-34],[14,-64,-36],[14,-64,-38],[14,-64,-40],[14,-66,-32],[14,-66,-34],[14,-66,-36],[14,-66,-38],[14,-66,-40],[14,-68,-34],[14,-68,-36],[14,-68,-38],[16,-50,-34],[16,-50,-36],[16,-50,-38],[16,-50,-40],[16,-52,-32],[16,-52,-34],[16,-52,-36],[16,-52,-38],[16,-52,-40],[16,-54,-30],[16,-54,-32],[16,-54,-34],[16,-54,-36],[16,-54,-38],[16,-54,-40],[16,-56,-30],[16,-56,-32],[16,-56,-34],[16,-56,-36],[16,-56,-38],[16,-56,-40],[16,-58,-30],[16,-58,-32],[16,-58,-34],[16,-58,-36],[16,-58,-38],[16,-58,-40],[16,-60,-30],[16,-60,-32],[16,-60,-34],[16,-60,-36],[16,-60,-38],[16,-60,-40],[16,-62,-30],[16,-62,-32],[16,-62,-34],[16,-62,-36],[16,-62,-38],[16,-62,-40],[16,-64,-30],[16,-64,-32],[16,-64,-34],[16,-64,-36],[16,-64,-38],[16,-64,-40],[16,-66,-32],[16,-66,-34],[16,-66,-36],[16,-66,-38],[16,-66,-40],[16,-68,-34],[16,-68,-36],[16,-68,-38],[18,-50,-38],[18,-52,-34],[18,-52,-36],[18,-52,-38],[18,-52,-40],[18,-54,-32],[18,-54,-34],[18,-54,-36],[18,-54,-38],[18,-54,-40],[18,-56,-32],[18,-56,-34],[18,-56,-36],[18,-56,-38],[18,-56,-40],[18,-58,-30],[18,-58,-32],[18,-58,-34],[18,-58,-36],[18,-58,-38],[18,-58,-40],[18,-60,-30],[18,-60,-32],[18,-60,-34],[18,-60,-36],[18,-60,-38],[18,-60,-40],[18,-62,-32],[18,-62,-34],[18,-62,-36],[18,-62,-38],[18,-62,-40],[18,-64,-32],[18,-64,-34],[18,-64,-36],[18,-64,-38],[18,-64,-40],[18,-66,-34],[18,-66,-36],[18,-66,-38],[18,-66,-40]]

df['coordinates'] = coordianteslist
df

,gradient1,gradient2,gradient3,gradient4,gradient5,gradient6,coordinates
1,-0.213402,-0.062360,0.055799,-0.049299,-0.021646,-0.059135,"[-10, -50, -34]"
2,-0.017965,-0.248732,0.026702,-0.005697,-0.062812,-0.055308,"[-10, -50, -36]"
3,-0.185052,0.059099,-0.043516,-0.015625,0.013516,-0.015204,"[-10, -62, -32]"
4,-0.025902,-0.002557,-0.030338,-0.053638,-0.001862,-0.017108,"[-10, -62, -34]"
5,0.246017,0.120548,0.000997,-0.031854,-0.006295,-0.017471,"[-10, -62, -36]"
6,-0.090228,0.049716,-0.032572,-0.022680,0.019769,-0.029338,"[-10, -64, -32]"
7,0.162496,0.028946,-0.082623,-0.052900,-0.005122,-0.016348,"[-10, -64, -34]"
8,0.313953,0.179834,-0.088148,-0.087836,-0.027382,0.009512,"[-10, -64, -36]"
9,-0.037485,0.094609,-0.043770,-0.006519,0.084929,-0.034090,"[-10, -66, -32]"
10,0.179097,0.068632,-0.082224,0.026799,0.056042,-0.062016,"[-10, -66, -34]"


In [32]:
import subprocess
for seed in df.index.tolist():
    if seed<167: # this means that seed came from left
        file='seed'+str(seed+52)+'_CerebralCortexNetworksMask_6mmSmoothing.nii'
    else:
        file='seedR'+str(seed+56-166)+'_CerebralCortexNetworksMask_6mmSmoothing.nii'
    subprocess.check_output("fslmaths "+file+" -mul "+str(df['gradient2'][seed])+" MulGrad2_"+file, shell=True);

In [41]:
# now repeat for gradient 1
import subprocess
for seed in df.index.tolist():
    if seed<167: # this means that seed came from left
        file='seed'+str(seed+52)+'_CerebralCortexNetworksMask_6mmSmoothing.nii'
    else:
        file='seedR'+str(seed+56-166)+'_CerebralCortexNetworksMask_6mmSmoothing.nii'
    subprocess.check_output("fslmaths "+file+" -mul "+str(df['gradient1'][seed])+" MulGrad1_"+file, shell=True);

In [ ]:
# add all files for gradient 1 and gradient 2 using fslmaths:
#fslmaths MulGrad1_seed53_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed54_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed55_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed56_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed57_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed58_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed59_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed60_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed61_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed62_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed63_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed64_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed65_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed66_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed67_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed68_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed69_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed70_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed71_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed72_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed73_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed74_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed75_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed76_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed77_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed78_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed79_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed80_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed81_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed82_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed83_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed84_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed85_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed86_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed87_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed88_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed89_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed90_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed91_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed92_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed93_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed94_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed95_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed96_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed97_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed98_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed99_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed100_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed101_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed102_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed103_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed104_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed105_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed106_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed107_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed108_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed109_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed110_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed111_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed112_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed113_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed114_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed115_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed116_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed117_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed118_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed119_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed120_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed121_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed122_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed123_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed124_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed125_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed126_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed127_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed128_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed129_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed130_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed131_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed132_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed133_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed134_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed135_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed136_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed137_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed138_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed139_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed140_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed141_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed142_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed143_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed144_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed145_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed146_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed147_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed148_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed149_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed150_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed151_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed152_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed153_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed154_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed155_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed156_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed157_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed158_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed159_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed160_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed161_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed162_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed163_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed164_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed165_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed166_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed167_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed168_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed169_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed170_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed171_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed172_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed173_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed174_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed175_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed176_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed177_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed178_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed179_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed180_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed181_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed182_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed183_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed184_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed185_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed186_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed187_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed188_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed189_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed190_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed191_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed192_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed193_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed194_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed195_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed196_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed197_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed198_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed199_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed200_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed201_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed202_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed203_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed204_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed205_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed206_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed207_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed208_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed209_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed210_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed211_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed212_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed213_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed214_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed215_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed216_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed217_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seed218_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR57_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR58_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR59_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR60_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR61_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR62_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR63_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR64_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR65_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR66_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR67_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR68_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR69_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR70_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR71_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR72_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR73_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR74_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR75_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR76_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR77_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR78_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR79_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR80_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR81_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR82_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR83_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR84_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR85_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR86_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR87_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR88_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR89_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR90_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR91_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR92_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR93_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR94_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR95_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR96_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR97_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR98_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR99_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR100_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR101_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR102_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR103_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR104_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR105_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR106_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR107_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR108_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR109_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR110_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR111_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR112_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR113_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR114_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR115_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR116_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR117_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR118_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR119_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR120_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR121_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR122_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR123_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR124_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR125_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR126_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR127_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR128_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR129_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR130_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR131_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR132_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR133_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR134_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR135_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR136_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR137_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR138_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR139_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR140_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR141_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR142_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR143_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR144_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR145_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR146_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR147_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR148_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR149_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR150_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR151_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR152_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR153_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR154_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR155_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR156_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR157_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR158_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR159_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR160_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR161_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR162_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR163_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR164_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR165_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR166_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR167_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR168_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR169_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR170_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR171_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR172_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR173_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR174_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR175_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR176_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR177_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR178_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR179_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR180_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR181_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR182_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR183_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR184_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR185_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR186_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR187_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR188_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR189_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR190_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR191_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR192_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR193_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR194_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR195_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR196_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR197_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR198_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR199_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR200_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR201_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR202_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR203_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR204_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR205_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR206_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR207_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR208_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR209_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR210_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR211_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR212_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR213_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR214_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR215_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR216_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR217_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR218_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR219_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR220_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR221_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR222_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR223_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR224_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR225_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR226_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR227_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR228_CerebralCortexNetworksMask_6mmSmoothing.nii -add MulGrad1_seedR229_CerebralCortexNetworksMask_6mmSmoothing.nii MulGrad1_AllAdded.nii

In [ ]:
# # map to surface with this:
# fslmaths MulGrad2_AllAdded.nii.gz -mul -1 MulGrad2_AllAdded_MulMinus1.nii.gz
# wb_command -volume-to-surface-mapping MulGrad2_AllAdded_MulMinus1.nii.gz S1200.L.inflated_MSMAll.32k_fs_LR.surf.gii Mul2Grad_MulMinus1_surfaceL.shape.gii -trilinear
# wb_command -volume-to-surface-mapping MulGrad2_AllAdded_MulMinus1.nii.gz S1200.R.inflated_MSMAll.32k_fs_LR.surf.gii Mul2Grad_MulMinus1_surfaceR.shape.gii -trilinear
# wb_command -volume-to-surface-mapping MulGrad1_AllAdded.nii.gz S1200.L.inflated_MSMAll.32k_fs_LR.surf.gii Mul1Grad_surfaceL.shape.gii -trilinear
# wb_command -volume-to-surface-mapping MulGrad1_AllAdded.nii.gz S1200.R.inflated_MSMAll.32k_fs_LR.surf.gii Mul1Grad_surfaceR.shape.gii -trilinear

# # multiply margulies by minus 1 to align orientation of gradients:
# wb_command -cifti-math 'x *(-1)' result_bilateralcortex_mulminus1.dscalar.nii -var x result_bilateralcortex.dscalar.nii